In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import RadiusNeighborsClassifier

remove_fight_island = False    #Removes fight island fights from consideration

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)





4793

In [6]:
#The fight island fights were messing with the established models.  This function most likely isn't needed anymore
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_no_fight_island = df[(df['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    #df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df = df_no_fight_island
    #df_train = df_train_no_fight_island

len(df)

4793

In [7]:
num_upcoming_fights = len(df_upcoming)

In [8]:
df = df_upcoming.append(df)
len(df)

4805

In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-e6abcb5e0e70>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-9-e6abcb5e0e70>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4793
4793
4793


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

R_fighter          B_fighter  R_odds  B_odds        R_ev  \
0     Islam Makhachev      Thiago Moises    -900     600   11.111111   
1       Marion Reneau        Miesha Tate     155    -180  155.000000   
2     Jeremy Stephens     Mateusz Gamrot     220    -275  220.000000   
3      Rodolfo Vieira   Dustin Stoltzfus    -210     175   47.619048   
4     Gabriel Benitez  Billy Quarantillo    -150     130   66.666667   
...               ...                ...     ...     ...         ...   
4788     Duane Ludwig      Darren Elkins    -155     135   64.516129   
4789      John Howard     Daniel Roberts    -210     175   47.619048   
4790   Brendan Schaub      Chase Gormley    -260     220   38.461538   
4791      Mike Pierce      Julio Paulino    -420     335   23.809524   
4792     Eric Schafer        Jason Brilz     140    -160  140.000000   

            B_ev       date                   location country Winner  ...  \
0     600.000000 2021-07-17     Las Vegas, Nevada, USA     USA    Red  ...   
1      55.555556 2021-07-17     Las Vegas, Nevada, USA     USA   Blue  ...   
2      36.363636 2021-07-17     Las Vegas, Nevada, USA     USA   Blue  ...   
3     175.000000 2021-07-17     Las Vegas, Nevada, USA     USA    Red  ...   
4     130.000000 2021-07-17     Las Vegas, Nevada, USA     USA   Blue  ...   
...          ...        ...                        ...     ...    ...  ...   
4788  135.000000 2010-03-21  Broomfield, Colorado, USA     USA   Blue  ...   
4789  175.000000 2010-03-21  Broomfield, Colorado, USA     USA    Red  ...   
4790  220.000000 2010-03-21  Broomfield, Colorado, USA     USA    Red  ...   
4791  335.000000 2010-03-21  Broomfield, Colorado, USA     USA    Red  ...   
4792   62.500000 2010-03-21  Broomfield, Colorado, USA     USA   Blue  ...   

      finish_round finish_round_time total_fight_time_secs  r_dec_odds  \
0              4.0              2:38                1058.0      -105.0   
1              3.0              1:53                 713.0       240.0   
2              1.0              1:05                  65.0       500.0   
3              3.0              1:54                 714.0       400.0   
4              3.0              3:40                 820.0       150.0   
...            ...               ...                   ...         ...   
4788           1.0              0:44                  44.0         NaN   
4789           1.0              2:01                 121.0         NaN   
4790           1.0              0:47                  47.0         NaN   
4791           3.0              5:00                 900.0         NaN   
4792           3.0              5:00                 900.0         NaN   

      b_dec_odds  r_sub_odds  b_sub_odds  r_ko_odds  b_ko_odds  label  
0         1000.0       275.0      1800.0      330.0     1600.0    0.0  
1          110.0       700.0       500.0     1100.0     1200.0    1.0  
2          150.0      1600.0      1100.0      450.0      180.0    1.0  
3          400.0      -110.0      1400.0      800.0      500.0    0.0  
4          350.0       600.0       850.0      350.0      500.0    1.0  
...          ...         ...         ...        ...        ...    ...  
4788         NaN         NaN         NaN        NaN        NaN    1.0  
4789         NaN         NaN         NaN        NaN        NaN    0.0  
4790         NaN         NaN         NaN        NaN        NaN    0.0  
4791         NaN         NaN         NaN        NaN        NaN    0.0  
4792         NaN         NaN         NaN        NaN        NaN    1.0  

[4793 rows x 120 columns]

R_fighter        B_fighter  R_odds  B_odds        R_ev  \
0       Cory Sandhagen     TJ Dillashaw    -195     165   51.282051   
1           Aspen Ladd    Macy Chiasson    -200     170   50.000000   
2       Kyler Phillips    Raulian Paiva    -260     210   38.461538   
3        Darren Elkins   Darrick Minner     130    -155  130.000000   
4     Miranda Maverick    Maycee Barber    -150     130   66.666667   
5          Mickey Gall  Jordan Williams     150    -170  150.000000   
6     Punahele Soriano    Brendan Allen    -110    -110   90.909091   
7   Nassourdine Imavov     Ian Heinisch     130    -150  130.000000   
8         Adrian Yanez      Randy Costa    -220     180   45.454545   
9           Julio Arce      Andre Ewell    -200     170   50.000000   
10      Sijara Eubanks       Elise Reed    -365     280   27.397260   
11       Diana Belbita     Hannah Goldy    -105    -115   95.238095   

          B_ev       date                location country Winner  ...  \
0   165.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
1   170.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
2   210.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
3    64.516129 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
4   130.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
5    58.823529 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
6    90.909091 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
7    66.666667 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
8   180.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
9   170.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
10  280.000000 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   
11   86.956522 2021-07-24  Las Vegas, Nevada, USA     USA    NaN  ...   

    finish_round finish_round_time total_fight_time_secs  r_dec_odds  \
0            NaN               NaN                   NaN       450.0   
1            NaN               NaN                   NaN       150.0   
2            NaN               NaN                   NaN      -105.0   
3            NaN               NaN                   NaN       275.0   
4            NaN               NaN                   NaN       120.0   
5            NaN               NaN                   NaN       400.0   
6            NaN               NaN                   NaN       555.0   
7            NaN               NaN                   NaN       250.0   
8            NaN               NaN                   NaN       350.0   
9            NaN               NaN                   NaN       110.0   
10           NaN               NaN                   NaN       120.0   
11           NaN               NaN                   NaN       180.0   

    b_dec_odds  r_sub_odds  b_sub_odds  r_ko_odds  b_ko_odds  label  
0        600.0       600.0      1000.0      120.0      350.0    NaN  
1        300.0       700.0      1400.0      275.0      700.0    NaN  
2        450.0       800.0      1200.0      350.0      800.0    NaN  
3        240.0       500.0       250.0      900.0      700.0    NaN  
4        275.0       600.0      1200.0     1000.0      400.0    NaN  
5        225.0       400.0      1100.0     1100.0      175.0    NaN  
6        450.0         NaN       225.0        NaN      700.0    NaN  
7        140.0       800.0      1000.0      700.0      450.0    NaN  
8        500.0      1000.0      1600.0     -110.0      450.0    NaN  
9        300.0       450.0      1400.0      600.0      700.0    NaN  
10       550.0       400.0      1800.0      275.0     1000.0    NaN  
11       140.0       900.0      1200.0      700.0      700.0    NaN  

[12 rows x 120 columns]

In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [16]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    #print(df_train)
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(ev_prepped_df)

In [17]:
with open('../data/production_models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
2
3
4
5
6
7
8
9
10
YES
11
12
YES
The model choice is 12
[['model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20', 'model_2021_06_24', 'model_2021_06_30', 'model_2021_07_21'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)', 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples

In [18]:

output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Cory Sandhagen vs. TJ Dillashaw
Cory Sandhagen has an EV of 51.28 on odds of -195.  They have 100.0 % chance of winning.

Aspen Ladd vs. Macy Chiasson
Aspen Ladd has an EV of 50.0 on odds of -200.  They have 100.0 % chance of winning.

Kyler Phillips vs. Raulian Paiva
Kyler Phillips has an EV of 38.46 on odds of -260.  They have 100.0 % chance of winning.

Darren Elkins vs. Darrick Minner
Darren Elkins has an EV of 130.0 on odds of 130.  They have 100.0 % chance of winning.

Miranda Maverick vs. Maycee Barber
Miranda Maverick has an EV of 66.67 on odds of -150.  They have 100.0 % chance of winning.

Mickey Gall vs. Jordan Williams
Mickey Gall has an EV of 150.0 on odds of 150.  They have 100.0 % chance of winning.

Punahele Soriano vs. Brendan Allen
Punahele Soriano has an EV of 90.91 on odds of -110.  They have 100.0 % chance of winning.

Nassourdine Imavov vs. Ian Heinisch
Nassourdine Imavov has an EV of 130.0 on odds of 130.  They have 100.0 % chance of winning.

Adrian Yanez vs. Ra

In [19]:
output_df

t1_name          t2_name  t1_odds  t2_odds  t1_prob  t2_prob  \
0       Cory Sandhagen     TJ Dillashaw     -195      165      1.0      0.0   
1           Aspen Ladd    Macy Chiasson     -200      170      1.0      0.0   
2       Kyler Phillips    Raulian Paiva     -260      210      1.0      0.0   
3        Darren Elkins   Darrick Minner      130     -155      1.0      0.0   
4     Miranda Maverick    Maycee Barber     -150      130      1.0      0.0   
5          Mickey Gall  Jordan Williams      150     -170      1.0      0.0   
6     Punahele Soriano    Brendan Allen     -110     -110      1.0      0.0   
7   Nassourdine Imavov     Ian Heinisch      130     -150      1.0      0.0   
8         Adrian Yanez      Randy Costa     -220      180      1.0      0.0   
9           Julio Arce      Andre Ewell     -200      170      1.0      0.0   
10      Sijara Eubanks       Elise Reed     -365      280      1.0      0.0   
11       Diana Belbita     Hannah Goldy     -105     -115      1.0      0.0   

    winner  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  
5      NaN  
6      NaN  
7      NaN  
8      NaN  
9      NaN  
10     NaN  
11     NaN

In [20]:
return_df = output_df[['t1_name', 't2_name', 't1_prob', 't2_prob']]

In [21]:
return_df

t1_name          t2_name  t1_prob  t2_prob
0       Cory Sandhagen     TJ Dillashaw      1.0      0.0
1           Aspen Ladd    Macy Chiasson      1.0      0.0
2       Kyler Phillips    Raulian Paiva      1.0      0.0
3        Darren Elkins   Darrick Minner      1.0      0.0
4     Miranda Maverick    Maycee Barber      1.0      0.0
5          Mickey Gall  Jordan Williams      1.0      0.0
6     Punahele Soriano    Brendan Allen      1.0      0.0
7   Nassourdine Imavov     Ian Heinisch      1.0      0.0
8         Adrian Yanez      Randy Costa      1.0      0.0
9           Julio Arce      Andre Ewell      1.0      0.0
10      Sijara Eubanks       Elise Reed      1.0      0.0
11       Diana Belbita     Hannah Goldy      1.0      0.0

In [22]:
output_df = return_df.rename(columns = {'t1_name': 'R_fighter', 't2_name': 'B_fighter', 't1_prob': 'R_prob', 't2_prob': 'B_prob'})

In [23]:
output_df.to_csv('../data/upcoming_probs.csv', index=False)